In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy

Loading all of the images out of the file system. Rescale the images to fit on a 0-1 scale, and also mess around with the training images in various ways to generate more synthetic data to train on. 

For reference, the resulting shape is (X, 256, 256, 1), which represents ([# images], [height], [width], [color channels])

In [65]:
training_data_generator = ImageDataGenerator(
    rescale=1.0/255,
    zoom_range=0.2,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05
)

test_data_generator = ImageDataGenerator(
    rescale=1.0/255
)

batch_size = 33

training_iterator = training_data_generator.flow_from_directory('Covid19-dataset/train', class_mode='categorical', color_mode='grayscale', batch_size=batch_size)
test_iterator = test_data_generator.flow_from_directory('Covid19-dataset/test', class_mode='categorical', color_mode='grayscale', batch_size=batch_size)


Found 251 images belonging to 3 classes.


Found 66 images belonging to 3 classes.


Build the actual model. It is a convolutional neural network with multiple convolution and pooling layers before the fully connected dense layers

Dropout layers were added to try to reduce a noticeable gap in the loss scores between training and test

In [77]:
model = Sequential()
model.add(tf.keras.Input(shape=(256, 256, 1)))
model.add(tf.keras.layers.Conv2D(10,5,activation='relu',padding='valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Conv2D(10,3,activation='relu',padding='valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 252, 252, 10)      260       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 126, 126, 10)      0         
 ng2D)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 126, 126, 10)      0         
                                                                 
 conv2d_29 (Conv2D)          (None, 124, 124, 10)      910       
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 62, 62, 10)        0         
 ng2D)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 62, 62, 10)      

Compile the model, and set the parameters for optimization, loss, and metrics

In [78]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy', 
    metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]
)


Fit the model

In [79]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    training_iterator, steps_per_epoch=training_iterator.samples/batch_size,
    epochs=50,
    validation_data=test_iterator,
    validation_steps=test_iterator.samples/batch_size,
    callbacks=[early_stopping]
)

Epoch 1/50


2024-01-26 13:07:19.038839: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_16/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 116ms/step - loss: 1.3905 - categorical_accuracy: 0.4821 - auc_16: 0.6508 - val_loss: 0.8109 - val_categorical_accuracy: 0.7121 - val_auc_16: 0.8869
Epoch 2/50
7/7 [==============================] - 1s 122ms/step - loss: 0.6517 - categorical_accuracy: 0.7211 - auc_16: 0.8848 - val_loss: 0.7348 - val_categorical_accuracy: 0.7424 - val_auc_16: 0.9070
Epoch 3/50
7/7 [==============================] - 1s 121ms/step - loss: 0.5940 - categorical_accuracy: 0.7410 - auc_16: 0.9019 - val_loss: 0.7552 - val_categorical_accuracy: 0.7273 - val_auc_16: 0.8713
Epoch 4/50
7/7 [==============================] - 1s 128ms/step - loss: 0.5343 - categorical_accuracy: 0.8008 - auc_16: 0.9232 - val_loss: 0.6546 - val_categorical_accuracy: 0.7879 - val_auc_16: 0.9231
Epoch 5/50
7/7 [==============================] - 1s 125ms/step - loss: 0.4648 - categorical_accuracy: 0.8088 - auc_16: 0.9405 - val_loss: 0.6766 - val_categorical_accuracy: 0.7121 - val_auc_16: 0.9028
E